In [1]:
import QDMPy.io as Qio
import QDMPy.fit as Qfit
import QDMPy.plot as Qplot
import QDMPy.bfield as Qbfield

import matplotlib
import matplotlib.pyplot as plt
import warnings

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

### Load raw data

In [3]:
options_dict = {
    # Argus: C:\\src\\QDMPY_test_image
    # Sam: /home/samsc/ResearchData/test_images/
    "base_dir": "/home/samsc/ResearchData/test_images/",
    "filepath": "ODMR - Pulsed_42",
    
    "additional_bins": 8,

    "system_name": "Zyla",

    "ROI": "Full", 
    "ROI_start": [10, 10],
    "ROI_end": [200,200],
    
    "AOI_1_start": [60, 145],
    "AOI_1_end": [70, 155], 
    "AOI_2_start": [20, 20],
    "AOI_2_end": [24, 24], 
    "AOI_3_start": [30, 20],
    "AOI_3_end": [32, 24], 
    "single_pixel_check": [90, 150],

    "normalisation": "div",
    
    "fit_backend": "scipyfit",
    "fit_backend_comparison": ["scipyfit"],
    "fit_pixels": True, 
    "force_fit": False, 
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

#     "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145], # ODMR 83
    "pos_guess": [2631, 2723, 2787, 2866, 2955, 3022, 3066, 3130], # ODMR 42
    "pos_range": 25,

    "amp_guess": -0.02,
    "amp_bounds": [-0.0300, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],
    
    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],

    "gpufit_tolerance": 1e-12,
    "gpufit_max_iterations": 25,
    "gpufit_estimator_id": "LSE"
}

In [4]:
options = Qio.load_options(options_dict=options_dict, check_for_prev_result=True)
Qplot.set_mpl_rcparams(options)
image, prelim_sweep_list = Qio.load_image_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = Qio.reshape_dataset(
        options, image, prelim_sweep_list
    )
Qplot.plot_ROI_PL_image(options, PL_image)

Qplot.plot_AOI_PL_images(options, PL_image_ROI)

Qplot.plot_AOI_spectra(options, sig, ref, sweep_list)

Qio.save_PL_data(options, PL_image, PL_image_ROI)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/systems.py:400: UserWarning: Option scipy_sub_threads was not recognised by the Zyla system.
  warnings.warn(f"Option {key} was not recognised by the {system.name} system.")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit AOI, ROI averages, single pixel

In [5]:
fit_model = Qfit.define_fit_model(options)
backend_ROI_results_lst = Qfit.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
Qplot.plot_ROI_avg_fits(options, backend_ROI_results_lst)

fit_result_collection_lst = Qfit.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, backend_ROI_results_lst)
Qplot.plot_AOI_spectra_fit(options, sig, ref, sweep_list, fit_result_collection_lst, fit_model)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit/_scipyfit.py:36: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit pixels

In [6]:
if options["found_prev_result"]:
    pixel_fit_params = Qfit.load_prev_fit_results(options)
    warnings.warn("Plotting previous fit results.")
elif options["fit_pixels"]:
    pixel_fit_params = Qfit.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
else:
    pixel_fit_params = None # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

Qio.save_pixel_fit_results(options, pixel_fit_params)

  0%|          | 0/21600 [00:00<?, ? PX/s]

### Plot (fit) parameters

In [7]:
Qplot.plot_param_images(options, fit_model, pixel_fit_params, "c")
Qplot.plot_param_images(options, fit_model, pixel_fit_params, "m")
Qplot.plot_param_images(options, fit_model, pixel_fit_params, "pos")
Qplot.plot_param_images(options, fit_model, pixel_fit_params, "amp")
Qplot.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load reference experiment - note, you can fit in this same notebook!

In [ ]:
ref_options_dir = None # if you want to use a reference experiment
ref_fit_params = Qio.load_reference_experiment_fit_results(options, ref_options=None, ref_options_dir=ref_options_dir)

### Calculate B_nvs, subtract reference

In [ ]:
sig_bnvs, sig_dshifts = Qbfield.get_bnvs_and_dshifts(pixel_fit_params)
ref_bnvs, ref_dshifts = Qbfield.get_bnvs_and_dshifts(ref_fit_params)

# TODO other bsubs e.g. subtract_blurred, subtract_outside_polygons etc. should be defined on any 2D array, do that later
bnvs = Qbfield.bnv_refsub(options, sig_bnvs, ref_bnvs)

Qio.save_bnvs_and_dshifts(options, "sig", sig_bnvs, sig_dshifts)
Qio.save_bnvs_and_dshifts(options, "sig_sub_ref", bnvs, None)

Qplot.plot_bnvs_and_dshifts(options, "sig", sig_bnvs, sig_dshifts)
Qplot.plot_bnvs_and_dshifts(options, "ref", ref_bnvs, ref_dshifts)
Qplot.plot_bnvs_and_dshifts(options, "sig_sub_ref", bnvs, [])

### Save and view options dict

In [8]:
Qio.save_options(options)
options

{'base_dir': '/home/samsc/ResearchData/test_images/',
 'filepath': '/home/samsc/ResearchData/test_images/ODMR - Pulsed_42',
 'custom_output_dir_prefix': None,
 'custom_output_dir_suffix': None,
 'custom_output_dir': None,
 'raw_pixel_size': 2.44140625e-07,
 'ignore_ref': False,
 'additional_bins': 8,
 'old_binning_convention': False,
 'system_name': 'Zyla',
 'ROI': 'Full',
 'ROI_start': [10, 10],
 'ROI_end': [200, 200],
 'remove_start_sweep': 1,
 'remove_end_sweep': 0,
 'normalisation': 'div',
 'single_pixel_check': [90, 150],
 'AOI_1_start': (60, 145),
 'AOI_1_end': (70, 155),
 'AOI_2_start': (20, 20),
 'AOI_2_end': (24, 24),
 'AOI_3_start': (30, 20),
 'AOI_3_end': (32, 24),
 'AOI_4_start': None,
 'AOI_4_end': None,
 'AOI_5_start': None,
 'AOI_5_end': None,
 'fit_backend': 'scipyfit',
 'fit_backend_comparison': ['scipyfit'],
 'force_fit': True,
 'fit_pixels': True,
 'scramble_pixels': True,
 'use_ROI_avg_fit_res_for_all_pixels': True,
 'fit_functions': {'linear': 1, 'lorentzian': 8},
